In [ ]:
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
from google.colab import drive
drive.mount('drive')
path = '/content/drive/MyDrive/idoven/'

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
from numpy import load

# Train
X_train = load(path+'X_train.npy')
y_train = load(path+'y_train.npy')
# Test
X_test = load(path+'X_test.npy')
y_test = load(path+'y_test.npy')

normal_indices = load(path+'normal_ecg_indices.npy')

In [ ]:
X_train_normal_samples = np.take(X_train, normal_indices, axis=0)

In [ ]:
X_train_normal_samples.shape

(8526, 1000, 12)

In [ ]:
sampling_rate=100

# Get the size of each sample
input_size = (1000,12)

In [ ]:
# Define la arquitectura del autoencoder
input_signal = keras.Input(shape=input_size)
encoded = layers.Dense(1024, activation='tanh')(input_signal)
encoded = layers.Dense(512, activation='tanh')(encoded)
encoded = layers.Dense(256, activation='tanh')(encoded)

decoded = layers.Dense(512, activation='tanh')(encoded)
decoded = layers.Dense(12, activation='tanh')(decoded)


In [ ]:
# Crea el modelo autoencoder
autoencoder = keras.Model(input_signal, decoded)

In [ ]:
# Compila el modelo
autoencoder.compile(optimizer='adam', loss='mse')

In [ ]:
from sklearn.model_selection import train_test_split
x_train_normal, x_val_normal, _, _ = train_test_split(X_train_normal_samples, X_train_normal_samples, test_size=0.2, random_state=42)

In [ ]:
x_train_normal.shape

(6820, 1000, 12)

In [ ]:
# Entrena el modelo
autoencoder.fit(x_train_normal, x_train_normal, epochs=10, batch_size=32, validation_data=(x_val_normal, x_val_normal))

Epoch 1/10
214/214 [==============================] - 861s 4s/step - loss: 0.0103 - val_loss: 0.0034
Epoch 2/10
214/214 [==============================] - 844s 4s/step - loss: 0.0095 - val_loss: 0.0042
Epoch 3/10
214/214 [==============================] - 850s 4s/step - loss: 0.0094 - val_loss: 0.0033
Epoch 4/10
214/214 [==============================] - 836s 4s/step - loss: 0.0096 - val_loss: 0.0033
Epoch 5/10
214/214 [==============================] - 836s 4s/step - loss: 0.0097 - val_loss: 0.0032
Epoch 6/10
214/214 [==============================] - 835s 4s/step - loss: 0.0094 - val_loss: 0.0035
Epoch 7/10
214/214 [==============================] - 838s 4s/step - loss: 0.0095 - val_loss: 0.0032
Epoch 8/10
214/214 [==============================] - 836s 4s/step - loss: 0.0095 - val_loss: 0.0035
Epoch 9/10
214/214 [==============================] - 842s 4s/step - loss: 0.0096 - val_loss: 0.0032
Epoch 10/10
214/214 [==============================] - 851s 4s/step - loss: 0.0096 - val_lo

In [ ]:
autoencoder.save("autoencoder_model")

In [ ]:

# Calcula el error de reconstrucción para las señales de prueba
reconstructed = autoencoder.predict(X_train_normal_samples)
mse = np.mean(np.power(X_train_normal_samples - reconstructed, 2), axis=1)

267/267 [==============================] - 340s 1s/step


In [ ]:
# Establece un umbral para la detección de anomalías
threshold = np.mean(mse) + 2 * np.std(mse)

In [ ]:
sample = X_train[89].reshape((1, 1000, 12))
anomaly_sample = autoencoder.predict(sample)

1/1 [==============================] - 0s 64ms/step


In [ ]:
mse = np.mean(np.power(sample - anomaly_sample, 2), axis=1)
print(mse)

[[0.00151972 0.00158307 0.00112884 0.00140036 0.0003386  0.0011389
  0.01683282 0.02036681 0.01498682 0.02587847 0.02474061 0.01705785]]
